In [1]:
# Import class
import os
import sys
parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))
class_dir = os.path.join(parent_dir, 'Scripts')
sys.path.insert(0, class_dir)
import datetime as dt
import scraper
import empirical
import pandas as pd

In [2]:
best_markets = scraper.get_best_player_markets() #dt.date(2019, 1, 31)

Started at 2019-03-02 10:51:33.693982
Got game links from SB

Got odds strings from https://www.sportsbet.com.au/betting/basketball-us/nba-matches/chicago-bulls-at-atlanta-hawks-4539462 (1/7)
Got odds strings from https://www.sportsbet.com.au/betting/basketball-us/nba-matches/charlotte-hornets-at-brooklyn-nets-4539463 (2/7)
Got odds strings from https://www.sportsbet.com.au/betting/basketball-us/nba-matches/portland-trail-blazers-at-toronto-raptors-4539461 (3/7)
Got odds strings from https://www.sportsbet.com.au/betting/basketball-us/nba-matches/washington-wizards-at-boston-celtics-4539464 (4/7)
Got odds strings from https://www.sportsbet.com.au/betting/basketball-us/nba-matches/new-orleans-pelicans-at-phoenix-suns-4539465 (5/7)
Got odds strings from https://www.sportsbet.com.au/betting/basketball-us/nba-matches/los-angeles-clippers-at-sacramento-kings-4539466 (6/7)
Got odds strings from https://www.sportsbet.com.au/betting/basketball-us/nba-matches/milwaukee-bucks-at-los-angeles-laker

## Show markets and order by bookie edge

In [3]:
bookie_pr_edge_list = []
for market in best_markets:
    bookie_pr_edge_list.append(market.calc_spread())

for edge, market in sorted(zip(bookie_pr_edge_list, best_markets), key=lambda t:t[0]):
    print('Bookie edge {:.2%}'.format(edge))
    print(market, end='\n\n')

Bookie edge -1.13%
milwaukee-bucks @ los-angeles-lakers - 02-Mar
Recorded 02-Mar 10:56
LeBron James - 8.5 AST
Overs  2.15 sportsbet
Unders  1.91 ladbrokes

Bookie edge 1.09%
milwaukee-bucks @ los-angeles-lakers - 02-Mar
Recorded 02-Mar 10:56
Khris Middleton - 4.5 AST
Overs   2.1 sportsbet
Unders  1.87 ladbrokes

Bookie edge 1.76%
chicago-bulls @ atlanta-hawks - 02-Mar
Recorded 02-Mar 10:52
Kris Dunn - 9.5 PTS
Overs  1.81 sportsbet
Unders  2.15 ladbrokes

Bookie edge 1.97%
washington-wizards @ boston-celtics - 02-Mar
Recorded 02-Mar 10:54
Marcus Morris - 13.5 PTS
Overs  1.88 sportsbet
Unders  2.05 ladbrokes

Bookie edge 3.65%
new-orleans-pelicans @ phoenix-suns - 02-Mar
Recorded 02-Mar 10:55
Julius Randle - 3.5 AST
Overs  2.15 ladbrokes
Unders  1.75 sportsbet

Bookie edge 4.50%
milwaukee-bucks @ los-angeles-lakers - 02-Mar
Recorded 02-Mar 10:56
Khris Middleton - 17.5 PTS
Overs  1.87 ladbrokes
Unders  1.96 sportsbet

Bookie edge 4.57%
milwaukee-bucks @ los-angeles-lakers - 02-Mar
Recorde

## Calculate theos for best markets

In [4]:
past_n_games = 40
edge_list = []
player_list = []
over_under_list = []
baseline_list = []
stat_list = []
odds_list = []
theo_list = []
bookie_edge_list = []
exchange_list = []
for p_mrkt in best_markets:
        
    try:
        # If unable to calculate theo (e.g. if player's name not correct), move onto next one
        # Calculate theo odds
        theo_odds_over, theo_odds_under = empirical.empirical_odds(p_mrkt.player, p_mrkt.stat, 
                                 p_mrkt.baseline, past_n_games)
    except:
        print('Unable to calculate theo for {} {}'.format(p_mrkt.player,
                                                      p_mrkt.stat))
        continue

    theo_pr_over = 1/theo_odds_over
    theo_pr_under = 1 - theo_pr_over

    # Calculate edge
    edge_over = theo_pr_over * p_mrkt.odds_over - 1
    edge_under = theo_pr_under * p_mrkt.odds_under - 1

    if edge_over > edge_under:
        best_edge = edge_over
        over_under = 'over'
        theo_list.append(round(theo_odds_over,2))
        odds_list.append(p_mrkt.odds_over)
        exchange_list.append(p_mrkt.overs_exchange)
    else:
        best_edge = edge_under
        over_under = 'under'
        theo_list.append(round(theo_odds_under, 2))
        odds_list.append(p_mrkt.odds_under)
        exchange_list.append(p_mrkt.unders_exchange)

    bookie_edge = 1/float(p_mrkt.odds_over) + 1/float(p_mrkt.odds_under) - 1
    
    bookie_edge_list.append(round(bookie_edge, 2))
    edge_list.append(round(best_edge, 2))
    player_list.append(p_mrkt.player)
    over_under_list.append(over_under)
    baseline_list.append(p_mrkt.baseline)
    stat_list.append(p_mrkt.stat)
    
# Create df, sort by edge and print
df_game_mrkts_edge = pd.DataFrame({
    'Edge':edge_list,
    'Spread':bookie_edge_list,
    'Player':player_list,
    'Over_under':over_under_list,
    'Baseline':baseline_list,
    'Stat':stat_list,
    'Odds':odds_list,
    'Theo':theo_list,
    'Exchange':exchange_list
}, columns = ['Edge', 'Spread', 'Player', 'Over_under', 'Baseline', 'Stat', 'Odds', 'Theo', 'Exchange'])

df_game_mrkts_edge = df_game_mrkts_edge.sort_values(by='Spread', ascending=True)
df_game_mrkts_edge

Unable to calculate theo for Moe Harkless PTS
Unable to calculate theo for Moe Harkless REB


,Edge,Spread,Player,Over_under,Baseline,Stat,Odds,Theo,Exchange
106,0.19,-0.01,LeBron James,under,8.5,AST,1.91,1.60,ladbrokes
105,0.22,0.01,Khris Middleton,under,4.5,AST,1.87,1.54,ladbrokes
0,0.13,0.02,Kris Dunn,over,9.5,PTS,1.81,1.60,sportsbet
39,-0.01,0.02,Marcus Morris,over,13.5,PTS,1.88,1.90,sportsbet
73,0.14,0.04,Julius Randle,under,3.5,AST,1.75,1.54,sportsbet
92,0.27,0.04,Khris Middleton,under,17.5,PTS,1.96,1.54,sportsbet
101,-0.03,0.05,LeBron James,under,9.5,REB,1.77,1.82,ladbrokes
45,0.07,0.05,Al Horford,under,4.5,AST,1.71,1.60,ladbrokes
71,0.28,0.05,Elfrid Payton,under,7.5,AST,1.71,1.33,ladbrokes
66,-0.02,0.06,Kyle Lowry,over,8.5,AST,2.06,2.11,sportsbet


## Record and save in pickle file

In [5]:
import pickle

if True:
    data_dir = os.path.join(parent_dir, 'Data')
    file_dir = os.path.join(data_dir, 'historic_markets.pickle')

    historic_markets = pickle.load(open(file_dir, 'rb'))

    market_today = [(dt.datetime.now(), best_markets)]

    historic_markets.extend(market_today)

    pickle.dump(historic_markets, open(file_dir, 'wb'))
    
    print('Markets saved at {}'.format(dt.datetime.now()))

Markets saved at 2019-03-02 11:05:15.627357
